In [1]:
import pandas as pd
from pyhive import hive
import re

In [2]:
# 하이브 연결
cursor = hive.connect('localhost').cursor()

In [3]:
cursor

In [6]:
# 실행할 SQL 쿼리
# 제약바이오주의 2019년 1월 4일 주가를 조회합니다.
sql = """
SELECT T1.STK_CD, T1.STK_NM, T1.SEC_NM, T1.EX_CD,
T2.DT, T2.O_PRC, T2.L_PRC, T2.H_PRC, T2.C_PRC, T2.VOL,
M5_PRC, M10_PRC, M20_PRC,
T3.FIN_ITM_VAL
FROM STOCK T1
INNER JOIN HISTORY_DT T2
ON (T2.STK_CD = T1.STK_CD)
INNER JOIN FINANCE_Y T3
ON (T2.STK_CD = T3.STK_CD)
WHERE T1.SEC_NM = '제약바이오'
AND T1.EX_CD = 'KP'
AND T2.DT = '2019-01-04'
AND T3.FIN_ITM_NM = '주당순이익'
AND T3.YY = '2018'
"""

In [7]:
# SQL 쿼리 실행 준비
cursor.execute(sql)

In [8]:
# SQL 쿼리를 실행하고 결과를 result에 저장
result = cursor.fetchall()

In [9]:
# SQL 쿼리 실행 결과 출력
result

[('000020',
  '동화약품',
  '제약바이오',
  'KP',
  '2019-01-04',
  Decimal('9100.000'),
  Decimal('9100.000'),
  Decimal('9700.000'),
  Decimal('9530.000'),
  Decimal('115336.000'),
  Decimal('9236.000'),
  Decimal('9090.000'),
  Decimal('9293.500'),
  Decimal('365.000')),
 ('000220',
  '유유제약',
  '제약바이오',
  'KP',
  '2019-01-04',
  Decimal('10050.000'),
  Decimal('9970.000'),
  Decimal('10300.000'),
  Decimal('10200.000'),
  Decimal('82844.000'),
  Decimal('10270.000'),
  Decimal('10365.000'),
  Decimal('10775.000'),
  Decimal('639.000')),
 ('000230',
  '일동홀딩스',
  '제약바이오',
  'KP',
  '2019-01-04',
  Decimal('12116.000'),
  Decimal('11830.000'),
  Decimal('12259.000'),
  Decimal('12211.000'),
  Decimal('5895.000'),
  Decimal('12306.600'),
  Decimal('12415.000'),
  Decimal('12202.350'),
  Decimal('-363.000')),
 ('000520',
  '삼일제약',
  '제약바이오',
  'KP',
  '2019-01-04',
  Decimal('19900.000'),
  Decimal('19650.000'),
  Decimal('20550.000'),
  Decimal('20550.000'),
  Decimal('38394.000'),
  Decimal('20

In [10]:
# 조회한 컬럼 정보 조회
cursor.description

[('t1.stk_cd', 'STRING_TYPE', None, None, None, None, True),
 ('t1.stk_nm', 'STRING_TYPE', None, None, None, None, True),
 ('t1.sec_nm', 'STRING_TYPE', None, None, None, None, True),
 ('t1.ex_cd', 'STRING_TYPE', None, None, None, None, True),
 ('t2.dt', 'DATE_TYPE', None, None, None, None, True),
 ('t2.o_prc', 'DECIMAL_TYPE', None, None, None, None, True),
 ('t2.l_prc', 'DECIMAL_TYPE', None, None, None, None, True),
 ('t2.h_prc', 'DECIMAL_TYPE', None, None, None, None, True),
 ('t2.c_prc', 'DECIMAL_TYPE', None, None, None, None, True),
 ('t2.vol', 'DECIMAL_TYPE', None, None, None, None, True),
 ('m5_prc', 'DECIMAL_TYPE', None, None, None, None, True),
 ('m10_prc', 'DECIMAL_TYPE', None, None, None, None, True),
 ('m20_prc', 'DECIMAL_TYPE', None, None, None, None, True),
 ('t3.fin_itm_val', 'DECIMAL_TYPE', None, None, None, None, True)]

In [11]:
desc = cursor.description

In [12]:
# 리스트의 행 열 방향을 반대로 바꿈
list(zip(*desc))

[('t1.stk_cd',
  't1.stk_nm',
  't1.sec_nm',
  't1.ex_cd',
  't2.dt',
  't2.o_prc',
  't2.l_prc',
  't2.h_prc',
  't2.c_prc',
  't2.vol',
  'm5_prc',
  'm10_prc',
  'm20_prc',
  't3.fin_itm_val'),
 ('STRING_TYPE',
  'STRING_TYPE',
  'STRING_TYPE',
  'STRING_TYPE',
  'DATE_TYPE',
  'DECIMAL_TYPE',
  'DECIMAL_TYPE',
  'DECIMAL_TYPE',
  'DECIMAL_TYPE',
  'DECIMAL_TYPE',
  'DECIMAL_TYPE',
  'DECIMAL_TYPE',
  'DECIMAL_TYPE',
  'DECIMAL_TYPE'),
 (None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 (None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 (None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 (None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 (True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
 

In [13]:
# 0번째 열 리턴 → 조회한 컬럼 이름
list(zip(*desc))[0]

('t1.stk_cd',
 't1.stk_nm',
 't1.sec_nm',
 't1.ex_cd',
 't2.dt',
 't2.o_prc',
 't2.l_prc',
 't2.h_prc',
 't2.c_prc',
 't2.vol',
 'm5_prc',
 'm10_prc',
 'm20_prc',
 't3.fin_itm_val')

In [14]:
# 조회한 컬럼 이름을 column_name에 대입
column_name = list(zip(*desc))[0]

In [15]:
# 대문자로 변환한 컬럼 이름을 다시 column_name에 대입
column_name = [element.upper() for element in column_name]

In [16]:
column_name

['T1.STK_CD',
 'T1.STK_NM',
 'T1.SEC_NM',
 'T1.EX_CD',
 'T2.DT',
 'T2.O_PRC',
 'T2.L_PRC',
 'T2.H_PRC',
 'T2.C_PRC',
 'T2.VOL',
 'M5_PRC',
 'M10_PRC',
 'M20_PRC',
 'T3.FIN_ITM_VAL']

In [17]:
# 정규 표현식 파트
## re.sub(): 특정 문자를 찾아서 삭제
## \S: 문자열
## \.: .
## . 앞의 문자열을 찾아서 삭제

# T3.FIN_ITM_VAL을 FIN_ITM_VAL로 변환하는 코드입니다.
re.sub('\S*\.', "", "T3.FIN_ITM_VAL")

'FIN_ITM_VAL'

In [18]:
column_name = [re.sub('\S*\.', "", element) for element in column_name]

In [19]:
column_name

['STK_CD',
 'STK_NM',
 'SEC_NM',
 'EX_CD',
 'DT',
 'O_PRC',
 'L_PRC',
 'H_PRC',
 'C_PRC',
 'VOL',
 'M5_PRC',
 'M10_PRC',
 'M20_PRC',
 'FIN_ITM_VAL']

In [20]:
# result를 DataFrame으로 변환
df = pd.DataFrame(result)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,000020,동화약품,제약바이오,KP,2019-01-04,9100.000,9100.000,9700.000,9530.000,115336.000,9236.000,9090.000,9293.500,365.000
1,000220,유유제약,제약바이오,KP,2019-01-04,10050.000,9970.000,10300.000,10200.000,82844.000,10270.000,10365.000,10775.000,639.000
2,000230,일동홀딩스,제약바이오,KP,2019-01-04,12116.000,11830.000,12259.000,12211.000,5895.000,12306.600,12415.000,12202.350,-363.000
3,000520,삼일제약,제약바이오,KP,2019-01-04,19900.000,19650.000,20550.000,20550.000,38394.000,20240.000,20155.000,20875.000,-1621.000
4,001630,종근당홀딩스,제약바이오,KP,2019-01-04,58600.000,57800.000,59800.000,58400.000,3773.000,60280.000,60320.000,60680.000,5481.000
5,002390,한독,제약바이오,KP,2019-01-04,28150.000,27400.000,28950.000,28000.000,151657.000,29910.000,30105.000,29350.000,590.000
6,002720,국제약품,제약바이오,KP,2019-01-04,4070.000,4070.000,4395.000,4275.000,50009.000,4220.000,4234.900,4308.550,128.000
7,003000,부광약품,제약바이오,KP,2019-01-04,20670.000,20583.000,21588.000,21413.000,391200.000,21316.800,20400.100,19394.900,3019.000
8,003060,에이프로젠제약,제약바이오,KP,2019-01-04,1863.000,1765.000,1883.000,1844.000,477730.000,1846.000,1833.100,1916.000,-12.000
9,003220,대원제약,제약바이오,KP,2019-01-04,13865.000,13685.000,14000.000,13955.000,29095.000,14153.000,14243.000,14369.000,1263.000


In [21]:
# 컬럼 이름 수정
df.columns = column_name
df

,STK_CD,STK_NM,SEC_NM,EX_CD,DT,O_PRC,L_PRC,H_PRC,C_PRC,VOL,M5_PRC,M10_PRC,M20_PRC,FIN_ITM_VAL
0,000020,동화약품,제약바이오,KP,2019-01-04,9100.000,9100.000,9700.000,9530.000,115336.000,9236.000,9090.000,9293.500,365.000
1,000220,유유제약,제약바이오,KP,2019-01-04,10050.000,9970.000,10300.000,10200.000,82844.000,10270.000,10365.000,10775.000,639.000
2,000230,일동홀딩스,제약바이오,KP,2019-01-04,12116.000,11830.000,12259.000,12211.000,5895.000,12306.600,12415.000,12202.350,-363.000
3,000520,삼일제약,제약바이오,KP,2019-01-04,19900.000,19650.000,20550.000,20550.000,38394.000,20240.000,20155.000,20875.000,-1621.000
4,001630,종근당홀딩스,제약바이오,KP,2019-01-04,58600.000,57800.000,59800.000,58400.000,3773.000,60280.000,60320.000,60680.000,5481.000
5,002390,한독,제약바이오,KP,2019-01-04,28150.000,27400.000,28950.000,28000.000,151657.000,29910.000,30105.000,29350.000,590.000
6,002720,국제약품,제약바이오,KP,2019-01-04,4070.000,4070.000,4395.000,4275.000,50009.000,4220.000,4234.900,4308.550,128.000
7,003000,부광약품,제약바이오,KP,2019-01-04,20670.000,20583.000,21588.000,21413.000,391200.000,21316.800,20400.100,19394.900,3019.000
8,003060,에이프로젠제약,제약바이오,KP,2019-01-04,1863.000,1765.000,1883.000,1844.000,477730.000,1846.000,1833.100,1916.000,-12.000
9,003220,대원제약,제약바이오,KP,2019-01-04,13865.000,13685.000,14000.000,13955.000,29095.000,14153.000,14243.000,14369.000,1263.000


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   STK_CD       25 non-null     object
 1   STK_NM       25 non-null     object
 2   SEC_NM       25 non-null     object
 3   EX_CD        25 non-null     object
 4   DT           25 non-null     object
 5   O_PRC        25 non-null     object
 6   L_PRC        25 non-null     object
 7   H_PRC        25 non-null     object
 8   C_PRC        25 non-null     object
 9   VOL          25 non-null     object
 10  M5_PRC       25 non-null     object
 11  M10_PRC      25 non-null     object
 12  M20_PRC      25 non-null     object
 13  FIN_ITM_VAL  25 non-null     object
dtypes: object(14)
memory usage: 2.9+ KB


In [23]:
# DT 컬럼을 datetime으로 변환합니다.
df["DT"] = pd.to_datetime(df["DT"])

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   STK_CD       25 non-null     object        
 1   STK_NM       25 non-null     object        
 2   SEC_NM       25 non-null     object        
 3   EX_CD        25 non-null     object        
 4   DT           25 non-null     datetime64[ns]
 5   O_PRC        25 non-null     object        
 6   L_PRC        25 non-null     object        
 7   H_PRC        25 non-null     object        
 8   C_PRC        25 non-null     object        
 9   VOL          25 non-null     object        
 10  M5_PRC       25 non-null     object        
 11  M10_PRC      25 non-null     object        
 12  M20_PRC      25 non-null     object        
 13  FIN_ITM_VAL  25 non-null     object        
dtypes: datetime64[ns](1), object(13)
memory usage: 2.9+ KB


In [26]:
# O_PRC 컬럼부터 마지막 컬럼까지 정수로 변환
df.loc[:, "O_PRC":] = df.loc[:, "O_PRC":].astype("int64")

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   STK_CD       25 non-null     object        
 1   STK_NM       25 non-null     object        
 2   SEC_NM       25 non-null     object        
 3   EX_CD        25 non-null     object        
 4   DT           25 non-null     datetime64[ns]
 5   O_PRC        25 non-null     object        
 6   L_PRC        25 non-null     object        
 7   H_PRC        25 non-null     object        
 8   C_PRC        25 non-null     object        
 9   VOL          25 non-null     object        
 10  M5_PRC       25 non-null     object        
 11  M10_PRC      25 non-null     object        
 12  M20_PRC      25 non-null     object        
 13  FIN_ITM_VAL  25 non-null     object        
dtypes: datetime64[ns](1), object(13)
memory usage: 2.9+ KB


In [28]:
df

,STK_CD,STK_NM,SEC_NM,EX_CD,DT,O_PRC,L_PRC,H_PRC,C_PRC,VOL,M5_PRC,M10_PRC,M20_PRC,FIN_ITM_VAL
0,000020,동화약품,제약바이오,KP,2019-01-04,9100,9100,9700,9530,115336,9236,9090,9293,365
1,000220,유유제약,제약바이오,KP,2019-01-04,10050,9970,10300,10200,82844,10270,10365,10775,639
2,000230,일동홀딩스,제약바이오,KP,2019-01-04,12116,11830,12259,12211,5895,12306,12415,12202,-363
3,000520,삼일제약,제약바이오,KP,2019-01-04,19900,19650,20550,20550,38394,20240,20155,20875,-1621
4,001630,종근당홀딩스,제약바이오,KP,2019-01-04,58600,57800,59800,58400,3773,60280,60320,60680,5481
5,002390,한독,제약바이오,KP,2019-01-04,28150,27400,28950,28000,151657,29910,30105,29350,590
6,002720,국제약품,제약바이오,KP,2019-01-04,4070,4070,4395,4275,50009,4220,4234,4308,128
7,003000,부광약품,제약바이오,KP,2019-01-04,20670,20583,21588,21413,391200,21316,20400,19394,3019
8,003060,에이프로젠제약,제약바이오,KP,2019-01-04,1863,1765,1883,1844,477730,1846,1833,1916,-12
9,003220,대원제약,제약바이오,KP,2019-01-04,13865,13685,14000,13955,29095,14153,14243,14369,1263


In [29]:
# STK_NM: 종목명 → 50개를 조회하겠습니다.
df["STK_NM"].head(50)

0        동화약품
1        유유제약
2       일동홀딩스
3        삼일제약
4      종근당홀딩스
5          한독
6        국제약품
7        부광약품
8     에이프로젠제약
9        대원제약
10       보령제약
11      우리들제약
12     녹십자홀딩스
13        녹십자
14       일양약품
15       광동제약
16    한올바이오파마
17       신풍제약
18       셀트리온
19       이연제약
20       한미약품
21      동아에스티
22        종근당
23     JW생명과학
24       일동제약
Name: STK_NM, dtype: object

In [30]:
# FIN_ITM_VAL 컬럼의 이름을 '주당순이익'으로 바꿈
df.rename(columns={"FIN_ITM_VAL":"주당순이익"}, inplace=True)

In [31]:
df

,STK_CD,STK_NM,SEC_NM,EX_CD,DT,O_PRC,L_PRC,H_PRC,C_PRC,VOL,M5_PRC,M10_PRC,M20_PRC,주당순이익
0,000020,동화약품,제약바이오,KP,2019-01-04,9100,9100,9700,9530,115336,9236,9090,9293,365
1,000220,유유제약,제약바이오,KP,2019-01-04,10050,9970,10300,10200,82844,10270,10365,10775,639
2,000230,일동홀딩스,제약바이오,KP,2019-01-04,12116,11830,12259,12211,5895,12306,12415,12202,-363
3,000520,삼일제약,제약바이오,KP,2019-01-04,19900,19650,20550,20550,38394,20240,20155,20875,-1621
4,001630,종근당홀딩스,제약바이오,KP,2019-01-04,58600,57800,59800,58400,3773,60280,60320,60680,5481
5,002390,한독,제약바이오,KP,2019-01-04,28150,27400,28950,28000,151657,29910,30105,29350,590
6,002720,국제약품,제약바이오,KP,2019-01-04,4070,4070,4395,4275,50009,4220,4234,4308,128
7,003000,부광약품,제약바이오,KP,2019-01-04,20670,20583,21588,21413,391200,21316,20400,19394,3019
8,003060,에이프로젠제약,제약바이오,KP,2019-01-04,1863,1765,1883,1844,477730,1846,1833,1916,-12
9,003220,대원제약,제약바이오,KP,2019-01-04,13865,13685,14000,13955,29095,14153,14243,14369,1263


In [32]:
# PER = (종가) / (주당순이익)
df["PER"] = df["C_PRC"] / df["주당순이익"]

In [34]:
df

,STK_CD,STK_NM,SEC_NM,EX_CD,DT,O_PRC,L_PRC,H_PRC,C_PRC,VOL,M5_PRC,M10_PRC,M20_PRC,주당순이익,PER
0,000020,동화약품,제약바이오,KP,2019-01-04,9100,9100,9700,9530,115336,9236,9090,9293,365,26.109589
1,000220,유유제약,제약바이오,KP,2019-01-04,10050,9970,10300,10200,82844,10270,10365,10775,639,15.962441
2,000230,일동홀딩스,제약바이오,KP,2019-01-04,12116,11830,12259,12211,5895,12306,12415,12202,-363,-33.639118
3,000520,삼일제약,제약바이오,KP,2019-01-04,19900,19650,20550,20550,38394,20240,20155,20875,-1621,-12.67736
4,001630,종근당홀딩스,제약바이오,KP,2019-01-04,58600,57800,59800,58400,3773,60280,60320,60680,5481,10.65499
5,002390,한독,제약바이오,KP,2019-01-04,28150,27400,28950,28000,151657,29910,30105,29350,590,47.457627
6,002720,국제약품,제약바이오,KP,2019-01-04,4070,4070,4395,4275,50009,4220,4234,4308,128,33.398438
7,003000,부광약품,제약바이오,KP,2019-01-04,20670,20583,21588,21413,391200,21316,20400,19394,3019,7.092746
8,003060,에이프로젠제약,제약바이오,KP,2019-01-04,1863,1765,1883,1844,477730,1846,1833,1916,-12,-153.666667
9,003220,대원제약,제약바이오,KP,2019-01-04,13865,13685,14000,13955,29095,14153,14243,14369,1263,11.049089


In [35]:
df[["STK_CD", "STK_NM", "PER"]]

,STK_CD,STK_NM,PER
0,000020,동화약품,26.109589
1,000220,유유제약,15.962441
2,000230,일동홀딩스,-33.639118
3,000520,삼일제약,-12.67736
4,001630,종근당홀딩스,10.65499
5,002390,한독,47.457627
6,002720,국제약품,33.398438
7,003000,부광약품,7.092746
8,003060,에이프로젠제약,-153.666667
9,003220,대원제약,11.049089


여기까지 주식 정보를 판다스 데이터프레임으로 만들었습니다. 이제 이 데이터를 머신러닝/딥러닝에 활용할 수 있습니다.

# 봉 차트 / 캔들 차트

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyhive import hive
import re
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [37]:
# 그래프 스타일 설정
plt.style.use("ggplot")

In [55]:
# SQL 쿼리
# 2019년 1월 1일 ~ 당해 1월 31일 삼성전자의 종가 거래량 조회
sql = """
SELECT T1.STK_CD, T1.DT, T1.O_PRC, T1.C_PRC, T1.L_PRC, T1.H_PRC, T1.VOL
FROM HISTORY_DT T1
WHERE T1.STK_CD = '005930'
AND T1.DT >= '2019-01-01'
AND T1.DT <= '2019-12-31'
"""

In [56]:
cursor.execute(sql)

In [57]:
result = cursor.fetchall()

In [58]:
result

[('005930',
  '2019-01-02',
  Decimal('39400.000'),
  Decimal('38750.000'),
  Decimal('38550.000'),
  Decimal('39400.000'),
  Decimal('7847664.000')),
 ('005930',
  '2019-01-03',
  Decimal('38300.000'),
  Decimal('37600.000'),
  Decimal('37450.000'),
  Decimal('38550.000'),
  Decimal('12471493.000')),
 ('005930',
  '2019-01-04',
  Decimal('37450.000'),
  Decimal('37450.000'),
  Decimal('36850.000'),
  Decimal('37600.000'),
  Decimal('14108958.000')),
 ('005930',
  '2019-01-07',
  Decimal('38000.000'),
  Decimal('38750.000'),
  Decimal('37800.000'),
  Decimal('38900.000'),
  Decimal('12748997.000')),
 ('005930',
  '2019-01-08',
  Decimal('38000.000'),
  Decimal('38100.000'),
  Decimal('37950.000'),
  Decimal('39200.000'),
  Decimal('12756554.000')),
 ('005930',
  '2019-01-09',
  Decimal('38650.000'),
  Decimal('39600.000'),
  Decimal('38300.000'),
  Decimal('39600.000'),
  Decimal('17452708.000')),
 ('005930',
  '2019-01-10',
  Decimal('40000.000'),
  Decimal('39800.000'),
  Decimal('39

In [59]:
df = pd.DataFrame(result)
df

,0,1,2,3,4,5,6
0,005930,2019-01-02,39400.000,38750.000,38550.000,39400.000,7847664.000
1,005930,2019-01-03,38300.000,37600.000,37450.000,38550.000,12471493.000
2,005930,2019-01-04,37450.000,37450.000,36850.000,37600.000,14108958.000
3,005930,2019-01-07,38000.000,38750.000,37800.000,38900.000,12748997.000
4,005930,2019-01-08,38000.000,38100.000,37950.000,39200.000,12756554.000
...,...,...,...,...,...,...,...
241,005930,2019-12-23,56100.000,55500.000,55100.000,56400.000,9839252.000
242,005930,2019-12-24,55600.000,55000.000,54800.000,55700.000,11868463.000
243,005930,2019-12-26,54700.000,55400.000,54400.000,55400.000,9645034.000
244,005930,2019-12-27,55700.000,56500.000,55500.000,56900.000,12313056.000


In [60]:
desc = cursor.description
desc

[('t1.stk_cd', 'STRING_TYPE', None, None, None, None, True),
 ('t1.dt', 'DATE_TYPE', None, None, None, None, True),
 ('t1.o_prc', 'DECIMAL_TYPE', None, None, None, None, True),
 ('t1.c_prc', 'DECIMAL_TYPE', None, None, None, None, True),
 ('t1.l_prc', 'DECIMAL_TYPE', None, None, None, None, True),
 ('t1.h_prc', 'DECIMAL_TYPE', None, None, None, None, True),
 ('t1.vol', 'DECIMAL_TYPE', None, None, None, None, True)]

In [61]:
column_name = list(zip(*desc))[0]

In [62]:
column_name

('t1.stk_cd',
 't1.dt',
 't1.o_prc',
 't1.c_prc',
 't1.l_prc',
 't1.h_prc',
 't1.vol')

In [63]:
column_name = [element.upper() for element in column_name]
column_name

['T1.STK_CD',
 'T1.DT',
 'T1.O_PRC',
 'T1.C_PRC',
 'T1.L_PRC',
 'T1.H_PRC',
 'T1.VOL']

In [64]:
column_name = [re.sub('\S*\.', "", element) for element in column_name]
column_name

['STK_CD', 'DT', 'O_PRC', 'C_PRC', 'L_PRC', 'H_PRC', 'VOL']

In [65]:
df.columns = column_name
df

,STK_CD,DT,O_PRC,C_PRC,L_PRC,H_PRC,VOL
0,005930,2019-01-02,39400.000,38750.000,38550.000,39400.000,7847664.000
1,005930,2019-01-03,38300.000,37600.000,37450.000,38550.000,12471493.000
2,005930,2019-01-04,37450.000,37450.000,36850.000,37600.000,14108958.000
3,005930,2019-01-07,38000.000,38750.000,37800.000,38900.000,12748997.000
4,005930,2019-01-08,38000.000,38100.000,37950.000,39200.000,12756554.000
...,...,...,...,...,...,...,...
241,005930,2019-12-23,56100.000,55500.000,55100.000,56400.000,9839252.000
242,005930,2019-12-24,55600.000,55000.000,54800.000,55700.000,11868463.000
243,005930,2019-12-26,54700.000,55400.000,54400.000,55400.000,9645034.000
244,005930,2019-12-27,55700.000,56500.000,55500.000,56900.000,12313056.000


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246 entries, 0 to 245
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   STK_CD  246 non-null    object
 1   DT      246 non-null    object
 2   O_PRC   246 non-null    object
 3   C_PRC   246 non-null    object
 4   L_PRC   246 non-null    object
 5   H_PRC   246 non-null    object
 6   VOL     246 non-null    object
dtypes: object(7)
memory usage: 13.6+ KB


In [67]:
df["DT"] = pd.to_datetime(df["DT"])

In [68]:
df.loc[:, "C_PRC":] = df.loc[:, "C_PRC":].astype("int64")

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246 entries, 0 to 245
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   STK_CD  246 non-null    object        
 1   DT      246 non-null    datetime64[ns]
 2   O_PRC   246 non-null    object        
 3   C_PRC   246 non-null    object        
 4   L_PRC   246 non-null    object        
 5   H_PRC   246 non-null    object        
 6   VOL     246 non-null    object        
dtypes: datetime64[ns](1), object(6)
memory usage: 13.6+ KB


In [70]:
# 봉 차트 그리기
candlestick = go.Candlestick(
    x=df['DT'], open=df["O_PRC"],
    high=df['H_PRC'], low=df['L_PRC'],
    close=df['C_PRC'],
    increasing_line_color='red',
    decreasing_line_color='blue'
)
fig = go.Figure(data=[candlestick])
fig.update_layout(
    title="2019년 삼성전자 주식 가격",
    yaxis_title='주식 가격'
)
fig.show()

c:\Users\tjk\AppData\Local\anaconda3\envs\bigdata_env\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [71]:
candlestick = go.Candlestick(
    x=df['DT'], open=df["O_PRC"],
    high=df['H_PRC'], low=df['L_PRC'],
    close=df['C_PRC'],
    increasing_line_color='red',
    decreasing_line_color='blue',
    showlegend=False
)
# 거래량 막대 그래프
volume_bars = go.Bar(
    x=df['DT'], y=df['VOL'],
    showlegend=False,
    marker={
        "color":"rgba(128,128,128,0.5)" # 색상을 회색으로 설정
    }
)
# 2줄 1칸으로 나눠서 그래프 그림
fig = make_subplots(rows=2, cols=1,
                    shared_xaxes=True,
                    subplot_titles=("2019년 삼성전자 거래 가격", "거래량")
                    )
# 1번째 줄 1번째 칸에 봉차트 출력
fig.add_trace(candlestick, row=1, col=1)
# 2번째 줄 1번째 칸에 거래량 막대 차트 출력
fig.add_trace(volume_bars, row=2, col=1)

fig.update_layout(height=800,
                  width=1200,
                  title_text="삼성전자 봉차트")
fig.show()

c:\Users\tjk\AppData\Local\anaconda3\envs\bigdata_env\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

